In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -q datasets

In [ ]:
PROMPT = "a photo of sks dog"
CLASS_PROMPT = "a photo of dog"
OUTPUT_DIR="dreambooth-dog"

CLASS_DIR="class-images"
DATASET_NAME = "jhaochenz/demo_dog"
PRETRAINED_MODEL = "CompVis/stable-diffusion-v1-4"

DATASET_DIR = "downloaded_images"
IS_FROM_HUGGINGFACE_HUB = True

In [ ]:
from datasets import load_dataset
import os

if IS_FROM_HUGGINGFACE_HUB :

  ds = load_dataset(DATASET_NAME)
  os.makedirs(DATASET_DIR, exist_ok=True)

  for idx, item in enumerate(ds['train']):
      image = item["image"]
      image.save(os.path.join(DATASET_DIR, f"{idx}.jpg"))

In [ ]:
!pip install -q accelerate>=0.16.0 torchvision  diffusers bitsandbytes fsspec==2025.3.2

In [ ]:
!accelerate config default

In [ ]:
!git clone https://github.com/docty/diffuser-training.git

In [ ]:
#!git clone https://huggingface.co/Docty/{MODEL_ID}

In [ ]:
!accelerate launch {os.getcwd()}/diffuser-training/train_dreambooth.py \
  --pretrained_model_name_or_path="$PRETRAINED_MODEL"  \
  --instance_data_dir="$DATASET_DIR" \
  --class_data_dir="$CLASS_DIR" \
  --output_dir="$OUTPUT_DIR" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="$PROMPT" \
  --class_prompt="$CLASS_PROMPT" \
  --resolution=128 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=2 --gradient_checkpointing \
  --use_8bit_adam \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --max_train_steps=800 \
  --resume_from_checkpoint='latest' \
  --checkpoints_total_limit=1 \
  --push_to_hub

# Inference

In [ ]:
from huggingface_hub.repocard import RepoCard
from diffusers import StableDiffusionPipeline
import torch

MODEL_ID = OUTPUT_DIR
hub_model_id = f"Docty/{MODEL_ID}"
card = RepoCard.load(hub_model_id)
base_model_id = card.data.to_dict()["base_model"]

pipe = StableDiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda") 

In [ ]:
prompt = "A photo of sks dog playing football"

image = pipe(prompt, num_inference_steps=50,  guidance_scale=7.5).images[0]
image